# Bibliotecas

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from joblib import dump

# Entendendo os Dados

In [2]:
tweets = pd.read_csv('Tweets_Mg.csv')
tweets.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
Created At,Sun Jan 08 01:22:05 +0000 2017,Sun Jan 08 01:49:01 +0000 2017,Sun Jan 08 01:01:46 +0000 2017,Wed Jan 04 21:43:51 +0000 2017,Mon Jan 09 15:08:21 +0000 2017
Text,���⛪ @ Catedral de Santo Antônio - Governador ...,"� @ Governador Valadares, Minas Gerais https:/...","�� @ Governador Valadares, Minas Gerais https:...",��� https://t.co/BnDsO34qK0,��� PSOL vai questionar aumento de vereadores ...
Geo Coordinates.latitude,NaN,-41.9333,-41.9333,NaN,NaN
Geo Coordinates.longitude,NaN,-18.85,-18.85,NaN,NaN
User Location,Brasil,NaN,NaN,NaN,NaN
Username,Leonardo C Schneider,Wândell,Wândell,Ana estudando,Emily
User Screen Name,LeoCSchneider,klefnews,klefnews,estudandoconcur,Milly777
Retweet Count,0,0,0,0,0
Classificacao,Neutro,Neutro,Neutro,Neutro,Negativo


# Preparação dos Dados

In [32]:
tweets[4]

'��� PSOL vai questionar aumento de vereadores e prefeito de BH na Justiça - Politica - Estado de Minas https://t.co/DMg7BGsek5'

In [3]:
classes = tweets ['Classificacao']
tweets = tweets['Text']

In [4]:
transformador = CountVectorizer()
transformador

CountVectorizer()

In [5]:
tabela_freq = transformador.fit_transform(tweets)
tabela_freq.shape

(8199, 13029)

In [6]:
tabela_freq[0].todense()

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [7]:
tweets[0]

'���⛪ @ Catedral de Santo Antônio - Governador Valadares/MG https://t.co/JSbKamIqUJ'

In [8]:
print(str(transformador.vocabulary_)[:100])

{'catedral': 2638, 'de': 3433, 'santo': 10434, 'antônio': 1645, 'governador': 5391, 'valadares': 118


In [12]:
tabela_freq[0].todense()[0, 2637:2640]

matrix([[0, 1, 0]])

# Modelo

In [13]:
modelo = MultinomialNB()
modelo.fit(tabela_freq, classes)
modelo

MultinomialNB()

In [15]:
testes = [
    "Eu amo lasanha", #positivo
    "Eu amo frango", #positivo
    "Tenho raiva de quem diz que odeia.", #negativo
    "O dia tá feio", #negativo
    "perdi o onibus", #negativo
    "tanto faz suco ou água", # neutro acertou
    "Nem fede nem cheira mesmo." #neutro
]

In [18]:
# modelo.predict([testes])

In [19]:
passo_1 = transformador.transform(testes)
passo_1

<7x13029 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [20]:
passo_2 = modelo.predict(passo_1)
passo_2

array(['Neutro', 'Neutro', 'Neutro', 'Neutro', 'Neutro', 'Neutro',
       'Negativo'], dtype='<U8')

In [21]:
pipe = Pipeline([('transformador', transformador), ('modelo', modelo)])
pipe

Pipeline(steps=[('transformador', CountVectorizer()),
                ('modelo', MultinomialNB())])

In [22]:
pipe.predict(testes)

array(['Neutro', 'Neutro', 'Neutro', 'Neutro', 'Neutro', 'Neutro',
       'Negativo'], dtype='<U8')

In [33]:
pipe.predict(['A maça questionar STF'])

array(['Negativo'], dtype='<U8')

# Avaliação

In [27]:
resultados = cross_val_predict(pipe, tweets, classes, cv = 10)
resultados

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype='<U8')

In [28]:
print(pd.crosstab(classes, resultados, rownames = ["Real"], colnames=["Predito"]))

Predito   Negativo  Neutro  Positivo
Real                                
Negativo      2277     160         9
Neutro         241    2050       162
Positivo        42     347      2911


In [29]:
(2277 + 2050 + 2911)/8199

0.8827905842175875

# Implantação

In [30]:
dump(pipe, 'pipeline.joblib')

['pipeline.joblib']

Referências
=== 

<https://www.kaggle.com/leandrodoze/sentiment-analysis-in-portuguese>  
<https://github.com/minerandodados/mdrepo>